<a href="https://colab.research.google.com/github/ZeyadSabbah/TrivagoRecommenderSystem/blob/master/FeatureSelection%26Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection & Modeling
## Mounting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem


## Loading Libraries & Datasets

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import random
import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
TrainDataFilepath = './Datasets/clean_data/Sets/train.csv'
TrainData = pd.read_csv(TrainDataFilepath)

## Features Correlation

In [0]:
#declaring features and label
features = TrainData.drop(columns='clickout').columns
label = ['clickout']

CorrMatrix = TrainData[features].corr()
sn.heatmap(CorrMatrix, annot=True)
plt.figure(figsize=(15, 10))
plt.show()

## Using SelectKBest

In [0]:
X_train = TrainData[features]
y_train = TrainData[label]

filter based --> colinearity between dependent variables and indepent (wrapper)
hypbrid --> combine two concepts together (outperforms)

In [0]:
bestfeatures = SelectKBest(score_func=chi2, k=15)
fit = bestfeatures.fit(X_train, y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Specs', 'Score']  #naming the dataframe columns
print(featureScores.nlargest(15,'Score'))   #print 15 best features

                     Specs         Score
4            item_duration  2.840552e+08
6        item_interactions  1.272761e+07
11      NumberInReferences  1.121516e+07
12        NumberAsClickout  3.023488e+06
13   NumberAsFinalClickout  2.303099e+06
1                item_rank  2.095303e+06
0                    price  6.917975e+05
5    item_session_duration  6.014721e+05
8                 top_list  5.040456e+05
17               MeanPrice  1.894017e+05
14  FClickoutToImpressions  1.024321e+05
10     NumberInImpressions  9.218344e+04
15   FClickoutToReferences  6.741030e+04
16     FClickoutToClickout  6.567854e+04
9       NumberOfProperties  6.459671e+04


Scores are high which means the features are relevant to the output, so all of the features will be taken in modeling at first, and then by removing the least important features, the performance of the model will be captured.

TrainData is ready for processing and modeling, while validation and test sets still need to be engineered. There is a ready function that will transform sets into the same form of TrainData.
## Scaling Features

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
('imputer', SimpleImputer(strategy="median")),
('std_scaler', StandardScaler()),
])

from sklearn.compose import ColumnTransformer
full_pipeline = ColumnTransformer([
("num", num_pipeline, list(X_train))
])

X_train_scaled = full_pipeline.fit_transform(X_train)

## Models

In [0]:
def print_results(results):
  print('Best Prams: {}\n'.format(results.best_params_))

  means = results.cv_results_['mean_test_score']
  stds = results.cv_results_['std_test_score']
  for mean, std, params in zip(means, stds, results.cv_results_['params']):
    print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [0]:
def random_search(clf, parameters, X, y):
  search_obj = RandomizedSearchCV(clf, parameters)
  fit_obj = search_obj.fit(X, y)
  best_clf = fit_obj.best_estimator_
  return best_clf

###Without Resampling

#### Logistic Regression

In [0]:
t1  = datetime.now()
lr = LogisticRegression()
parameters = {
    'C':[0.01, 0.1, 1, 10]
}

cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(X_train_scaled, y_train.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'LR_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

Best Prams: {'C': 10}

0.965 (+/-0.0) for {'C': 0.01}
0.965 (+/-0.0) for {'C': 0.1}
0.965 (+/-0.0) for {'C': 1}
0.965 (+/-0.0) for {'C': 10}
Time taken :  0:25:42.234537


#### Random Forest

Searching for the right hyperparameters takes long time if the whole train set is used, that's why random sample will be taken from train set and will be provided in the random search.

In [0]:
X_train_scaled = pd.DataFrame(X_train_scaled)
X_train_scaled_sample = X_train_scaled.sample(frac=0.05, random_state=0)
y_train_sample = y_train.sample(frac=0.05, random_state=0)

In [0]:
t1 = datetime.now()
rf = RandomForestClassifier()
parameters = {
    'n_estimators':[5, 50, 250],
    'max_depth':[2, 4, 8, 16, 32]
}
best_clf_grid = random_search(rf, parameters, X_train_scaled_sample, y_train_sample.values.ravel())
print(best_clf_grid)
rf = best_clf_grid.fit(X_train_scaled, y_train.values.ravel())
joblib.dump(rf, './models/RF_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Time taken :  5:30:32.162484


In [0]:
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train.values.ravel())
joblib.dump(rf, './models/RF_model.pkl')

#### XGBoost

In [0]:
t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

best_clf_grid = random_search(gb, parameters, X_train_scaled_sample, y_train_sample.values.ravel())
print(best_clf_grid)

gb = best_clf_grid.fit(X_train_scaled, y_train.values.ravel())
joblib.dump(gb, './models/XGBoost_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

### With SMOTE

In [0]:
sm = SMOTE(random_state = 0, ratio = 1)
X_SM, y_SM = sm.fit_sample(X_train_scaled, y_train)

#### Logistic Regression

In [0]:
t1  = datetime.now()
lr = LogisticRegression()
parameters = {
    'C':[0.01, 0.1, 1, 10]
}

cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(X_SM_train, y_SM_train)

print_results(cv)
joblib.dump(cv.best_estimator_, './modelsSMOTE/LR_modelSMOTE.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

#### Random Forest

Only the same length of the original training set will be randomly selected, so it would not take long time training

In [0]:
X_SM = np.load('./Datasets/clean_data/SMOTE_X_y/X_SM.npy')
y_SM = np.load('./Datasets/clean_data/SMOTE_X_y/y_SM.npy')

X_SM = pd.DataFrame(X_SM)
y_SM = pd.DataFrame(y_SM)

X_SM_train = X_SM.sample(frac=0.5, random_state=0)
y_SM_train = y_SM.sample(frac=0.5, random_state=0)

X_SM_train_sample = X_SM_train.sample(frac=0.01, random_state=1)
y_SM_train_sample = y_SM_train.sample(frac=0.01, random_state=1)

In [0]:
t1 = datetime.now()
rf = RandomForestClassifier()
parameters = {
    'n_estimators':[5, 50, 250],
    'max_depth':[2, 4, 8, 16, 32]
}
best_clf_grid = random_search(rf, parameters, X_SM_train_sample, y_SM_train_sample)
print(best_clf_grid)

rf = best_clf_grid.fit(X_SM_train, y_SM_train)
joblib.dump(rf, './modelsSMOTE/RF_modelSMOTE.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Time taken :  2:03:05.524921


#### XGBoost

In [0]:
t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

best_clf_grid = random_search(gb, parameters, X_SM_sample, y_SM_sample)
gb = best_clf_grid.fit(X_SM_train, y_SM_train)
joblib.dump(gb, './modelsSMOTE/XGBoost_modelSMOTE.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

In [0]:
gb = GradientBoostingClassifier()
gb.fit(X_SM_sample, y_SM_sample)
joblib.dump(gb, './modelsSMOTE/XGBoostSMOTE.pkl')

['./modelsSMOTE/XGBoostSMOTE.pkl']

### With Undersampling

In [0]:
ClickoutLen = len(y_train[y_train.clickout==1])
NotClickoutIndices = y_train[y_train.clickout==0].index
ClickoutIndices = y_train[y_train.clickout==1].index
NotClickoutRandomIndices = np.random.choice(NotClickoutIndices, ClickoutLen, replace=False)
UnderSampleIndices = np.concatenate([ClickoutIndices, NotClickoutRandomIndices])

XUnderSample = []
yUnderSample = []
for Index in UnderSampleIndices.tolist():
  XUnderSample.append(X_train_scaled[Index].tolist())
  yUnderSample.append([y_train.clickout.values[Index]])
XUndderSample = np.array(XUnderSample)
yUnderSample = np.array(yUnderSample)

In [0]:
np.save('./Datasets/clean_data/UnderSampleXy/XUnderSample.npy', XUnderSample)
np.save('./Datasets/clean_data/UnderSampleXy/yUnderSample.npy', yUnderSample)

In [0]:
XUnderSample = np.load('./Datasets/clean_data/UnderSampleXy/XUnderSample.npy')
yUnderSample = np.load('./Datasets/clean_data/UnderSampleXy/yUnderSample.npy')


#### Logistic Regression

In [0]:
t1  = datetime.now()
lr = LogisticRegression()
parameters = {
    'C':[0.01, 0.1, 1, 10]
}

cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(XUnderSample, yUnderSample)

print_results(cv)
joblib.dump(cv.best_estimator_, './modelsUnderSampling/LR_modelUndersampling.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

Best Prams: {'C': 0.1}

0.799 (+/-0.002) for {'C': 0.01}
0.799 (+/-0.002) for {'C': 0.1}
0.799 (+/-0.002) for {'C': 1}
0.799 (+/-0.002) for {'C': 10}
Time taken :  0:02:32.136969


In [0]:
lr = LogisticRegression()
param_grid = {
    'C': [0.01, 0.1, 1, 10]
}
opt = BayesSearchCV(
    lr,
    param_grid
)
opt.fit(XUnderSample, yUnderSample)
opt.best_params_

#### Random Forest

In [0]:
XUnderSample = pd.DataFrame(XUnderSample)
yUnderSample = pd.DataFrame(yUnderSample)
XUnderSample_sample = XUnderSample.sample(frac=0.1, random_state=0)
yUnderSample_sample = yUnderSample.sample(frac=0.1, random_state=0)

NameError: ignored

In [0]:
t1 = datetime.now()
rf = RandomForestClassifier()
parameters = {
    'n_estimators':[5, 50, 250],
    'max_depth':[2, 4, 8, 16, 32]
}
best_clf_grid = random_search(rf, parameters, X_train_scaled_sample, y_train_sample.values.ravel())
print(best_clf_grid)
t2 = datetime.now()
t2 - t1

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


datetime.timedelta(0, 662, 908882)

In [0]:
t1 = datetime.now()
rf = best_clf_grid.fit(X_train_scaled, y_train.values.ravel())
joblib.dump(rf, './modelsUnderSampling/RF_modelUnderSampling.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

In [0]:
rf = RandomForestClassifier()
rf.fit(XUnderSample, yUnderSample)
joblib.dump(rf, './modelsUnderSampling/RF_modelUnderSampling.pkl')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


['./modelsUnderSampling/RF_modelUnderSampling.pkl']

In [0]:
rf.predict(X_train.iloc[0].values)

array([1.62000000e+02, 1.00000000e+00, 8.00000000e+00, 1.02500000e+03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.60000000e+01,
       1.00000000e+00, 5.80000000e+01, 5.84000000e+02, 4.79000000e+02,
       2.50000000e+01, 1.10000000e+01, 1.88356164e-02, 2.29645094e-02,
       4.40000000e-01, 1.94595890e+02, 1.18407534e+01])

#### XGBoost

In [0]:
t1 = datetime.now()
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators':[5, 50, 250, 500],
    'max_depth':[1, 3, 5, 7, 9],
    'learning_rate':[0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(XUnderSample, yUnderSample.values.ravel())

print_results(cv)
joblib.dump(cv.best_estimator_, 'XGB_model.pkl')
t2 = datetime.now()
print('Time taken : ', (t2 - t1))

In [0]:
gb = GradientBoostingClassifier()
gb.fit(XUnderSample, yUnderSample)
joblib.dump(gb, './modelsUnderSampling/XGBoostUnderSampling.pkl')

['./models/XGBoost_model.pkl']